<div>
<img src="https://www.ibm.com/blogs/bluemix/wp-content/uploads/2017/02/NLU.png", width=170, height=170, align='right'>  
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/51/IBM_logo.svg/640px-IBM_logo.svg.png", width = 90, height = 90, align = 'right', style="margin:left:2px 25px">
</div>

# Culture Transformation Jam 主题分析
本文演示基于LDA模型对不同问题进行主题分析的方法。<br>
This notebook runs on Python 3.5 with Spark 2.1.

## 内容列表

1.[实现目标](#goal)  
2.[准备工作](#pre)   
　　2.1 [库安装](#library)  
　　2.2 [分析文件导入](#data)  
3.[数据处理](#process)  
　　3.1 [根据首次分词结果调整jieba词库](#jieba)  
　　3.2 [数据清洗,过滤掉所有不符合规则的词](#clean)  
4.[数据探索](#basic)  
5.[模型构建](#model)  
　　5.1 [构建LDA主题模型](#lda)  
　　5.2 [问题对应的主题统计](#topic)  
　　5.3 [问题的核心主题词提取](#key)

<a id='goal'></a>
# 1. 根据调查问题，实现以下目标：

- 统计问题中不同词语的频次，对调查重点有初步了解
- 将所有问题分类为几大主题，比较该调查对不同主题的关注度
- 获得每一个问题的关键主题词，对每一个问题有更深刻认识

<a id='pre'></a>
# 2. 准备工作

<a id='library'></a>
### 2.1 库安装
包括：nltk、numpy、pandas、lda、jieba、codecs、sklearn

In [2]:
#coding:utf-8
import nltk

In [3]:
# 如果目标文件不存在，去掉这个注释即可安装文件
# nltk.download()

In [17]:
# load the required libraries
# ! pip install lda
import numpy as np
import pandas as pd
import lda
import jieba
import jieba.posseg
import codecs
from sklearn.feature_extraction.text import CountVectorizer
from io import StringIO
import requests
import json
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

<a id='data'></a>
### 2.2 分析文件导入

部分问题截图

In [19]:
# The code was removed by DSX for sharing.

,﻿Question
0,Do you like our new Agile Open Office?
1,CDL办公室管理的挑战：工作日有小朋友跟着家长来上班。一方面，学校放假期间小朋友不能自己呆在...
2,工程师的日常生活里，你觉得甚么最需要去改变？(e.g. 工作态度，社交活动，生活习惯，创新思...
3,"Dear IBMer, 你觉得你能够做些什么来让IBM重新在业界走上巅峰？(In your ..."
4,除了项目以外， 你一周学习新技术的时间有多少？比如：一周1个小时，一周4个小时，一周10个小...


<a id='process'></a>
# 3. 数据处理

<a id='jieba'></a>
### 3.1 根据首次分词结果调整jieba词库 

eg.“卡拉ok”、“马克笔”作为名词加入词库，分别替换原来的“卡拉” “ok”和“马克” “笔”

In [6]:
# 读取自定义的分词词库
add_list = np.array(pd.read_csv(get_object_storage_file('MyHackathon', 'add_list.csv')))
# 修改jieba现有词库
for item in add_list:
    jieba.add_word(item[0],tag=item[1])

Building prefix dict from the default dictionary ...
Dumping model to file cache /gpfs/fs01/user/sf31-bf990ab5b3efba-4276a445e7a2/notebook/tmp/jieba.cache
Loading model cost 1.398 seconds.
Prefix dict has been built succesfully.


<a id='clean'></a>
### 3.2 数据清洗,过滤掉所有不符合规则的词

- 过滤常用停用词,如 ！、@、#……0、1、2……
- 因为主题多为名词，过滤所有非名词

In [7]:
# 读取停用词表
com_stop_word=np.array(pd.read_csv(get_object_storage_file('MyHackathon', 'com_stop_words.csv')))
stop_list=np.array(pd.read_csv(get_object_storage_file('MyHackathon', 'stop_list.csv')))

# 过滤停用词并筛选出名词
clean_words=[]
ps = PorterStemmer()
for line in question:
    seg = jieba.posseg.cut(line[0])
    for j in seg:
        if j.flag!='eng'and j.word not in com_stop_word and j.word not in stop_list and j.flag.startswith('n') :   
            clean_words.append(j.word)
        elif j.flag=='eng' and j.word not in stopwords.words('english') and j.word not in com_stop_word and j.word not in stop_list:
            eng_tag=nltk.pos_tag(nltk.word_tokenize(j.word))
            for w,pos in eng_tag:
                if pos.startswith('NN'):

                    clean_words.append(w)  
# 得到干净的分词结果
corpus=[]                   
for line in question:
    seg_generator = jieba.cut(line[0])
    #筛出stop word
    seg_list = [i for i in seg_generator if i in clean_words]
    seg_list = r' '.join(seg_list)
    corpus.append(seg_list.strip())
print ('\n数据处理前后的3个问题\n')
print (question[0])
print ('->',corpus[0],'\n')
print (question[4])
print ('->',corpus[4],'\n')
print (question[5])
print ('->',corpus[5],'\n')


数据处理前后的3个问题

['Do you like our new Agile Open Office?']
-> Agile Office 

['除了项目以外， 你一周学习新技术的时间有多少？比如：一周1个小时，一周4个小时，一周10个小时，等等？']
-> 项目 学习 技术 时间 

['IBM要转型成功，你认为是技术更重要，还是满足客户需求更重要？']
-> 转型 成功 技术 客户需求 



<a id='basic'></a>
# 4. 对数据进行基本的探索 

### 对问题进行词频统计，生成词云

颜色越深，字体越大，表示该词在所有问题中出现的次数越多

In [8]:
import brunel
freq=''
for line in corpus:
    freq+=line
freq = pd.DataFrame(freq.split(" "),columns=['TYPE_WORDS'])
freq_froup = freq.groupby("TYPE_WORDS").size().reset_index()
freq_froup.columns = ["TYPE_WORDS","COUNT"]
freq_froup_large = freq_froup[freq_froup.COUNT > 1]
%brunel data('freq_froup_large') cloud x(TYPE_WORDS) size(COUNT:400%) color(COUNT:blues) sort(TYPE_WORDS) title(''):: width=600, height=300

<IPython.core.display.Javascript object>

分析:
- 本次调查对“小朋友”讨论的最多
- 对于“技术”、“问题”和“活动”也被提及多次

<a id='model'></a>
# 5. 模型构建

<a id='lda'></a>
### 5.1 构建LDA主题模型

LDA(Latent Dirichlet Allocation)是一种文档生成模型。它认为一篇文章是有多个主题的，而每个主题又对应着不同的词。
一篇文章的构造过程，首先是以一定的概率选择某个主题，然后再在这个主题下以一定的概率选出某一个词，这样就生成了这篇文章的一个词。
LDA使用词袋模型构造成词向量，利用上述文档生成的逆过程，根据一篇得到的文章，去寻找出这篇文章的主题，以及这些主题对应的词。

#### 参数设置：topic_number,alpha,beta

- 经验表明，对于90多个简单的问题，主题数量为10多个更合适
- 经过进一步实验发现,主题数量为10、15、20时的准确率分别为70%、75%、72%左右，于是在这个任务中topic_number设为15
- alpha和beta两个超参数，可以用gridSearch或其它度量模型的指标(主题间的相似度等)来找到最优解
- 但是经验表明，alpha取50/topic_num+1,beta取0.001往往能取到很好的效果

In [9]:
#词频矩阵
stop_words=[]
stop_list=np.array(pd.read_csv(get_object_storage_file('MyHackathon', 'stop_list.csv')))
for item in stop_list:
    stop_words.append(item[0])
vectorizer = CountVectorizer(stop_words=stop_words)

#将文本转为词频矩阵
X = vectorizer.fit_transform(corpus)
weight = X.toarray()

#建立模型,15个主题,迭代1500次,根据经验将超参数αβ分别设为50/T+1、0.001
topic_num=15
model=lda.LDA(n_topics=topic_num, n_iter=1500, random_state=1,alpha=(50.0/topic_num)+1,eta=0.001)
model.fit(X)

INFO:lda:n_documents: 93
INFO:lda:vocab_size: 203
INFO:lda:n_words: 333
INFO:lda:n_topics: 15
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -3758
INFO:lda:<10> log likelihood: -2903
INFO:lda:<20> log likelihood: -2879
INFO:lda:<30> log likelihood: -2877
INFO:lda:<40> log likelihood: -2893
INFO:lda:<50> log likelihood: -2875
INFO:lda:<60> log likelihood: -2896
INFO:lda:<70> log likelihood: -2873
INFO:lda:<80> log likelihood: -2889
INFO:lda:<90> log likelihood: -2872
INFO:lda:<100> log likelihood: -2887
INFO:lda:<110> log likelihood: -2873
INFO:lda:<120> log likelihood: -2897
INFO:lda:<130> log likelihood: -2873
INFO:lda:<140> log likelihood: -2897
INFO:lda:<150> log likelihood: -2875
INFO:lda:<160> log likelihood: -2881
INFO:lda:<170> log likelihood: -2880
INFO:lda:<180> log likelihood: -2872
INFO:lda:<190> log likelihood: -2868
INFO:lda:<200> log likelihood: -2890
INFO:lda:<210> log likelihood: -2876
INFO:lda:<220> log likelihood: -2891
INFO:lda:<230> log likelihood: -2880
INFO:ld

<a id='topic'></a>
### 5.2 问题对应的主题统计

#### 根据主题-单词的confidence，取前n个关键词描述该主题

In [10]:
# 主题-单词分布矩阵
topic_word = model.topic_word_ 
# 3个词语描述一个主题
n = 3
word = vectorizer.get_feature_names()
topic_words_list = []
for i, topic_dist in enumerate(topic_word):
    # 每个主题下词语按confidence从大到小排序，取前n个词来描述该主题
    topic_words = np.array(word)[np.argsort(topic_dist)][:-(n+1):-1]
    topic_words_list.append(' '.join(topic_words))

print("\n主题 - 单词分布  \n")
for index , value in enumerate(topic_words_list):
    index+=1
    print("topic%d: "%index, value)


主题 - 单词分布  

topic1:  社交活动 要求 批量
topic2:  开发 company 工程师
topic3:  工作 skill enablement
topic4:  活动 transformation 导向
topic5:  感觉 工作环境 范畴
topic6:  api ideas defect
topic7:  转型 项目 原因
topic8:  cdl workspace 家长
topic9:  公司 产品 创新
topic10:  小朋友 转变 文化
topic11:  agile 团队 办法
topic12:  建议 tvt team
topic13:  改变 openwhisk 同事
topic14:  技术 学习 产品经理
topic15:  问题 成功 culture


#### 统计每个主题在问题中涉及的频数

In [12]:
import numpy as np  
# 文档-主题分布矩阵
doc_topic = model.doc_topic_ 
# 找到与问题相关的topic_num个主题
topic_num=2
doc_topic_list = []
count=np.zeros((15))
for index , value in enumerate(doc_topic):
    # 每个问题下主题按confidence从大到小排序，获得该问题主要涉及的topic_num个主题
    topics=np.argsort(value)[:-(topic_num + 1):-1]
    for topic in topics:
        count[topic]+=1

# 根据频数从小到大排序
index=np.array(count).argsort()
count.sort()
topics=range(len(count))
topic_num=[]
for i in index:
    topic_num.append(topics[i])

# 主题频数图
topic_pd = pd.DataFrame(count,columns=['COUNT'])
topic_pd['TOPIC'] = ["Topic_" + str(x+1) for x in topic_num]
topic_pd['TOPIC_NUM'] = [-x for x in range(len(count))]
%brunel data('topic_pd') bar x(TOPIC) y(COUNT) color(COUNT:blues) label(COUNT) style('.label{fill:white;text-shadow:none}') sort(TOPIC_NUM) effect(enter:2000)  title("主题频数图")  axes(y:2000:grid:"数量",x:6:"主题"):: width=600, height=300

<IPython.core.display.Javascript object>

分析：
- 从主题频数图中可以看出：本次调查对主题15和14讨论的最多
- 结合“主题-单词”分布可知，调查围绕问题、技术、成功、学习等话题展开

<a id='key'></a>
### 5.3 问题的核心主题词提取 

#### 结合文档-主题和主题-单词的confidence，取前3个作为问题的核心主题词

In [13]:
# 每个问题取前2个主题
topic_num=2
# 每个主题取前3个单词
word_num=3
doc_topic = model.doc_topic_
topic_word= model.topic_word_ 
sav=[]
# 将文档-主题和主题-问题confidence相乘，从大到小排序，取前doc_num个词来描述该问题
for index , value in enumerate(doc_topic):
    topics=np.argsort(value)[:-(topic_num + 1):-1]
    word_idx=[]
    word_list=[]
    for topic_idx in topics:
        w=np.argsort(topic_word[topic_idx])[:-(word_num + 1):-1]
        for idx in w: 
            word_idx.append(idx)
            word_list.append(topic_word[topic_idx][idx]*value[topic_idx])
    # 每个问题取前3个主题词
    doc_num=3
    doc_word=''
    word_list = np.argsort(word_list)[:-(doc_num + 1):-1]
    for i in word_list:
        doc_word+=' '+np.array(word)[word_idx[i]]      
    
    # 将问题和关键词匹配，保存
    line=[]
    line.append(question[i])
    line.append(doc_word)
    line=np.array(line)
    sav.append(line)
df = pd.DataFrame(sav)
df.to_csv("que_key.csv",encoding='utf-8',index=False)

#### 图形化展示

In [14]:
df['QUESTION_INDEX']= ["Q"+ str(x) for x in df.index]
if len(df.iloc[0]) < 4:
    df.columns=['QUESTION','TOPIC_VALUE','QUESTION_INDEX']
else:
    df.columns=['QUESTION','TOPIC_VALUE','QUESTION_INDEX','TOPIC_WORD']
#主题第一个单词,用作主归类
df['TOPIC_WORD'] = [x.split(" ")[1] for x in df.TOPIC_VALUE]

In [15]:
%%brunel data('df') x(TOPIC_WORD,QUESTION_INDEX) color(TOPIC_WORD) treemap label(TOPIC_VALUE) tooltip(#all) 
    style('.label {font-size:6pt}') legends(none)
:: width=2000, height=1500

<IPython.core.display.Javascript object>

Tips：
- 每一个框代表了一个问题，框内为该问题的3个关键主题词
- 鼠标移动到某个框内，可查看问题内容、序号、最关键的主题词和前3个主题词
- 根据每个问题的最关键主题词分为”工作“、”成功“、”技术“、”感觉“、”项目“、”产品“、”小朋友“等几个大色块，同色方框表示这些问题具有相同的最关键主题词
- 每种颜色包含的问题越多、面积越大，表示该主题词在问题中出现的越多

分析:
- 从上图中可以看出，以”技术“为最关键主题词的问题是最多的，可见谈论技术的问题占比最大，关于成功的问题次之

***
### 作者
**颜 晓超** 是在IBM从事数据分析的高级分析师，有丰富的针对企业客户开发和提供各类数据分析服务经验

<div><br><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/51/IBM_logo.svg/640px-IBM_logo.svg.png" width = 200 height = 200>
</div><br>